前面的C，我们成功地学习了37个手写汉字。但是，那个Nerual network的DNN参数，是拍脑袋想的。我们需要再精调一下。



In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

IMAGE_SIZE = 64
FULL_CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)
    
train_data = DataSetLoader(data_dir='../data/train_/')
print ('Train data loaded.')

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

# Build 2 layer DNN classifier
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[1024, 1024, 1024, 1024, 1024],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=CHARSET_SIZE,
    dropout=0.1,
    model_dir="../dfs/checkpoint/dnn7_model_a"
)

# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(train_data)[0]},
    y=input(train_data)[1],
    num_epochs=None,
    batch_size=50,
    shuffle=True
)

classifier.train(input_fn=train_input_fn, steps=200000)
print ('Train done, begin to test ...')
test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

# Evaluate accuracy
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

Now processing path:  ../data/train_/00037
Train data loaded.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../dfs/checkpoint/dnn7_model_a', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn7_model_a/model.ckpt-76100
INFO:tensorflow:Saving checkpoints for 76101 into ../dfs/checkpoint/dnn7_model_a/model.ckpt.
INFO:tensorflow:loss = 0.119615, step = 76101
INFO:tensorflow:global_step/sec: 8.0875
INFO:tensorflow:loss = 0.11229, step = 76201 (12.367 sec)
INFO:tensorflow:global_step/sec: 8.24939
INFO:tensorflow:loss = 0.348018, step = 76301 (12.127 sec)
INFO:tensorflow:global_step/sec: 8.35475
INFO:tensorflow:loss = 4.61195, step = 76401 (11.965 sec)
INFO:tensorflo

上面还真好使：
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn5_model/model.ckpt-20000
INFO:tensorflow:Saving checkpoints for 20001 into ../dfs/checkpoint/dnn5_model/model.ckpt.
INFO:tensorflow:loss = 5.79399, step = 20001

从上次的20000步开始。从这里看来，即前使用浏览器训练时没有了响应，后台也是保存了的。就是前台失去了联系。
下面我们做点有趣的：手工观察测试情况。参考https://colab.research.google.com/drive/1eSVbFPcHHt1BbSKwzdclRDeLIIx_5ZOR#scrollTo=JkbP89CJd1bC （linear-regression-D.ipynb）

In [1]:
# from: https://www.kaggle.com/jeffcarp/example-save-and-load-a-tensorflow-model

import tensorflow as tf
import numpy as np
import os
import random
from numpy import array

from skimage.io import imread, imsave
from skimage.color import rgb2gray
from skimage.transform import resize

from tensorflow.contrib.learn.python.learn.estimators import run_config
from tensorflow.contrib.training.python.training import hparam

IMAGE_SIZE = 64
# CHARSET_SIZE = 3755
CHARSET_SIZE = 37

def input(dataset):
    return dataset.images, dataset.labels

class DataSetLoader:
    def __init__(self, data_dir):
        # Set CHARSET_SIZE to a small value if available computation power is limited.
        truncate_path = data_dir + ('%05d' % CHARSET_SIZE)
        print('Now processing path: ', truncate_path)
        image_names = []
        for root, sub_folder, file_list in os.walk(data_dir):
            if root < truncate_path:
                image_names += [os.path.join(root, file_path) for file_path in file_list]
        random.shuffle(image_names)
        self.labels = [int(file_name[len(data_dir):].split(os.sep)[0]) for file_name in image_names]
        images_rgb = [imread(file_name) for file_name in image_names]
        image_resized = [resize(image, (IMAGE_SIZE, IMAGE_SIZE)) for image in images_rgb]
        self.images = [rgb2gray(item) for item in image_resized]
        
        # convert list to numpy array
        self.images = array(self.images)
        self.labels = array(self.labels)

# Specify feature
feature_columns = [tf.feature_column.numeric_column("x", shape=[IMAGE_SIZE, IMAGE_SIZE])]

def make_estimator(model_dir):
    config = run_config.RunConfig(model_dir=model_dir)

    return tf.estimator.DNNClassifier (
        config=config,
        feature_columns=feature_columns,
        hidden_units=[1024, 1024, 1024, 1024, 1024],
        optimizer=tf.train.AdamOptimizer(1e-4),
        n_classes=CHARSET_SIZE,
        dropout=0.1
    )

MODEL_DIR = "../dfs/checkpoint/dnn7_model_a"
model_from_checkpoint = make_estimator(MODEL_DIR)

test_data = DataSetLoader(data_dir='../data/test_/')
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    y=input(test_data)[1],
    num_epochs=1,
    shuffle=False
)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(test_data)[0]},
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy
accuracy_score = model_from_checkpoint.evaluate(input_fn=test_input_fn)["accuracy"]
print("\nTest Accuracy: {0:f}%\n".format(accuracy_score*100))

# manually test
predictions = list(model_from_checkpoint.predict(input_fn=predict_input_fn))
predictions_value_array = list(predictions)

import matplotlib.pyplot as plt

def test_n_hanzi(predictions_values, index):
    # Show the image of the input data
    image_data_array = test_data.images[index]
    image_data = image_data_array.reshape((IMAGE_SIZE, IMAGE_SIZE))
    plt.imshow(image_data)
    plt.show()
    
    # show the prediction result
    print ('Prediction label is: ', np.argmax(predictions_values[index]["probabilities"]))
    
    # Show the lable
    print ("Real label is: %d"%(test_data.labels[index]))

for i in range(37):
    test_n_hanzi(predictions_value_array, i)    

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f78586387f0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '../dfs/checkpoint/dnn7_model_a'}
Now processing path:  ../data/test_/00037
INFO:tensorflow:Starting evaluation at 2018-04-21-02:13:37
INFO:tensorflow:Restoring parameters from ../dfs/checkpoint/dnn7_model_a/model.ckpt-276100
INFO:tensorflow:Finished evaluation at 2018-04-21-02:13:38
INFO:tensorflow:Saving dict for global step 276100: accuracy = 0.814932, average_loss = 1.6523

Prediction label is:  19
Real label is: 19


Prediction label is:  26
Real label is: 26


Prediction label is:  0
Real label is: 0


Prediction label is:  20
Real label is: 20


...

Prediction label is:  15
Real label is: 15


Prediction label is:  26
Real label is: 26


Prediction label is:  24
Real label is: 17


Prediction label is:  15
Real label is: 15


Prediction label is:  6
Real label is: 2
